In [1]:
from inference import OllamaClient
client = OllamaClient(base_url="http://localhost:11434")
model="gemma3:4b"
models = client.list_models()
assert model in [m["name"] for m in models["models"]], f"Model {model} not found in Ollama server"
client.warmup_model(model=model)
# client.warmup_model(model=evaluator_model)

Model gemma3:4b warmed up successfully.


In [2]:
# prompt="What is the meaning of Cheese?"
# res = client.generate(model=model, prompt=prompt, temperature=0.7, max_tokens=50)
# print(res)
# # res = client.generate(model=evaluator_model, prompt=prompt, temperature=0.7, max_tokens=50)
# # print(res)

In [3]:
import pandas as pd

splits = {'train': 'main/train-00000-of-00001.parquet', 'test': 'main/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/openai/gsm8k/" + splits["train"])

c:\Users\Owner\anaconda3\envs\cs401r\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare PRewriteDataset and trainer

The cells below show how to wrap the loaded DataFrame into `PRewriteDataset`,
create an Ollama evaluator and the `PRewritePPOTrainer`, and run a short
training run for testing. Adjust `max_steps` for real training.

In [4]:
# Create PRewriteDataset from the already-loaded DataFrame `df`
from load_datasets import PRewriteDataset, prepare_df_to_jsonl
# reserve 10% as dev (paper used 10% for GSM8K)
ds = PRewriteDataset(df, split="train", dev_fraction=0.1, seed=42)
print('train size:', len(ds.get_split_df('train')) )
if ds.get_split_df('dev') is not None:
    print('dev size:', len(ds.get_split_df('dev')))

train size: 6726
dev size: 747


In [5]:
# Load finetune model from Hugging Face (will save locally if not present)
from finetune import load_from_hf
import torch
torch.cuda.is_available()


model_id = "Qwen/Qwen3-4B-Instruct-2507"
save_dir = "models/Qwen3-4B-Instruct-local"

# This will try to load from `save_dir` first; if missing it downloads and saves there.
tokenizer, hf_model = load_from_hf(model_id, save_to=save_dir, trust_remote_code=True)
print('Model/tokenizer available at:', save_dir)

# Expose the local path for the trainer to use as `base_model` if desired
base_model_local = save_dir


Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 158.48it/s]

Model/tokenizer available at: models/Qwen3-4B-Instruct-local
